In [71]:
import pandas as pd
import requests
from sklearn.linear_model import LinearRegression

# -------- CONFIG --------
API_KEY = "b76ab9b14abba7b3a9a6f86d5c021414"  # <-- Replace with your API key
COUNTIES = COUNTIES = {
    'Dublin': (53.3498, -6.2603),
    'Galway': (53.2707, -9.0568),
    'Carlow': (52.8365, -6.9341),
    'Clare': (52.8436, -9.0031),
    'Cork': (51.8985, -8.4756),
    'Cavan': (53.9908, -7.3606),
    'Westmeath': (53.5333, -7.3667),
    'Mayo': (53.7667, -9.5000),
    'Sligo': (54.2706, -8.4716),
    'Meath': (53.6050, -6.6556),
    'Tipperary': (52.4735, -8.1618),
    'Donegal': (54.6542, -8.1096),
    'Wexford': (52.3365, -6.4629),
    'Roscommon': (53.6333, -8.1833)
}
TODAY = pd.to_datetime("today").date()

# -------- 1. Load Historical Data --------
df = pd.read_csv("counties_with_data_2015_2022.csv")
df['date'] = pd.to_datetime(df['date'], format="%d-%m-%Y %H:%M")
daily_df = df.groupby([df['date'].dt.date, 'county']).agg({
    'temp': 'mean',
    'rain': 'sum'
}).reset_index().rename(columns={'date': 'day'})

# -------- 2. Get Today's Weather from OpenWeatherMap --------
def get_weather(lat, lon):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {
        'temp': data['main']['temp'],
        'rain': data.get('rain', {}).get('1h', 0.0)  # mm in last hour
    }

today_rows = []
for county, (lat, lon) in COUNTIES.items():
    weather = get_weather(lat, lon)
    today_rows.append({
        'county': county,
        'day': TODAY,
        'temp': weather['temp'],
        'rain': weather['rain']
    })

today_df = pd.DataFrame(today_rows)

# -------- 3. Combine Historical + Today's Data --------
combined_df = pd.concat([daily_df, today_df], ignore_index=True)
combined_df = combined_df.sort_values(['county', 'day']).reset_index(drop=True)

# -------- 4. Create Targets (t+1 to t+7) --------
forecast_horizon = 5
for i in range(1, forecast_horizon + 1):
    combined_df[f'temp_t+{i}'] = combined_df.groupby('county')['temp'].shift(-i)
    combined_df[f'rain_t+{i}'] = combined_df.groupby('county')['rain'].shift(-i)

train_df = combined_df.dropna()

# -------- 5. Train Model & Predict --------
features = ['temp', 'rain']
forecast_results = {}

for county in train_df['county'].unique():
    county_train = train_df[train_df['county'] == county]
    X = county_train[features]
    y_temp = county_train[[f'temp_t+{i}' for i in range(1, 6)]]
    y_rain = county_train[[f'rain_t+{i}' for i in range(1, 6)]]

    temp_model = LinearRegression().fit(X, y_temp)
    rain_model = LinearRegression().fit(X, y_rain)

    latest_input = today_df[today_df['county'] == county][features]
    if not latest_input.empty:
        temp_pred = temp_model.predict(latest_input)[0]
        rain_pred = rain_model.predict(latest_input)[0]

        forecast_results[county] = {
            'temperature_next_5_days': temp_pred.round(2).tolist(),
            'rainfall_next_5_days': rain_pred.round(2).tolist()
        }

# -------- 6. Show Forecast --------
for county, forecast in forecast_results.items():
    print(f"\n📍 Forecast for {county} (from {TODAY}):")
    for i in range(5):
        print(f"Day +{i+1}: Temp = {forecast['temperature_next_5_days'][i]}°C, Rain = {forecast['rainfall_next_5_days'][i]} mm")




📍 Forecast for Carlow (from 2025-04-18):
Day +1: Temp = 10.64°C, Rain = 1.91 mm
Day +2: Temp = 10.64°C, Rain = 1.98 mm
Day +3: Temp = 10.63°C, Rain = 2.04 mm
Day +4: Temp = 10.59°C, Rain = 2.03 mm
Day +5: Temp = 10.59°C, Rain = 2.11 mm

📍 Forecast for Cavan (from 2025-04-18):
Day +1: Temp = 8.89°C, Rain = 2.39 mm
Day +2: Temp = 8.98°C, Rain = 2.57 mm
Day +3: Temp = 9.01°C, Rain = 2.61 mm
Day +4: Temp = 9.01°C, Rain = 2.59 mm
Day +5: Temp = 9.02°C, Rain = 2.66 mm

📍 Forecast for Clare (from 2025-04-18):
Day +1: Temp = 9.18°C, Rain = 2.32 mm
Day +2: Temp = 9.34°C, Rain = 2.56 mm
Day +3: Temp = 9.41°C, Rain = 2.63 mm
Day +4: Temp = 9.41°C, Rain = 2.69 mm
Day +5: Temp = 9.44°C, Rain = 2.74 mm

📍 Forecast for Cork (from 2025-04-18):
Day +1: Temp = 11.87°C, Rain = 6.87 mm
Day +2: Temp = 11.85°C, Rain = 7.49 mm
Day +3: Temp = 11.79°C, Rain = 7.59 mm
Day +4: Temp = 11.72°C, Rain = 7.88 mm
Day +5: Temp = 11.74°C, Rain = 8.27 mm

📍 Forecast for Donegal (from 2025-04-18):
Day +1: Temp = 8.18°C, 

Error: Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.
